## Add extension to example resources

- read file from folder
- convert to dict or model
- add extension
    with data from file name
    optionally user input - future
- save file back to folder

In [ ]:
from pathlib import Path
from json import dumps, loads
from requests import post


In [ ]:
in_path = Path(r'\\ERICS-AIR-2\ehaas\Documents\FHIR\US-Core-R4\input\examples')


ext_1 =  [
      {
        "url": "http://hl7.org/fhir/StructureDefinition/instance-name",
        "valueString": 'my_name'
      },
      {
        "url": "http://hl7.org/fhir/StructureDefinition/instance-description",
        "valueMarkdown": 'my_descr'
      }
    ]
    
    
print(ext_1)  

def convert(r):

    fhir_test_server = 'http://test.fhir.org/r4'
    #fhir_test_server = 'http://hapi.fhir.org/baseR4'
    #fhir_test_server = 'http://wildfhir4.aegis.net/fhir4-0-1'   
    headers = {
    'Accept':'application/fhir+json',
    'Content-Type':'application/fhir+xml'
    }
    params = {}
    r = post(f'{fhir_test_server}/$convert', params = params, headers = headers, data = r)
    print(r.status_code)
    # view  output
    # return (r.json()["text"]["div"])
    return r.json()

def entitle(name):
    x = name.split('-')
    return(' '.join(i.title() for i in x if i != "example"))

def enname(name):
    x = name.split('-')
    return(' '.join(i for i in x if i != "example"))

def p_name(f):
    #p = loads(f.open().read())
    p=f
    try:
        profile_url = (p['meta']['profile'][0])
        return profile_url.split('/')[-1]
    except KeyError:
        return p['resourceType']

files = [x for x in Path(in_path).iterdir() if x.is_file() and x.name.endswith('json')]
#files = [x for x in Path(in_path).iterdir() if x.is_file() and x.name.endswith('xml')] # Get xml and convert and rename to json
for f in files:

    dict_data = loads(f.open().read()) #dict
    #print(data)

    try:
     print(dumps(dict_data['meta'], indent=3 ))
    except KeyError:
        dict_data['meta'] = {}
    
    print(f, type(f), f.stem, entitle(f.stem) + ' Example')
    ext_1[0]['valueString'] = entitle(f.stem) + ' Example'
    print(ext_1)
    ext_1[1]['valueMarkdown'] = f'This is a {enname(f.stem)} example for the *{entitle(p_name(dict_data))} Profile*.'
    dict_data['meta']['extension'] = ext_1
    print(dumps(dict_data['meta'], indent=3))
    print(dumps(dict_data, indent=3))
    print('===========================================================')
    f.open('w').write(dumps(dict_data, indent=3))
    '''
    # change to json
    print(dumps(convert(data),indent=3))
    dict_data = convert(data)
    f.open('w').write(dumps(dict_data, indent=3))
    print(f.parent,type(f.parent), f.name,type(f.name), f.suffix)
    new_name = f"{str(f.parent)}\{f.stem}.json"
    print(str(f),new_name)
    f.rename(new_name)
    '''    
